In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [3]:
# Define target url. Search for houses in Copenhagen and surroundings, boligtype = Rent
url = "https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/"
response = requests.get(url)

In [4]:
response.status_code

200

In [5]:
soup = BeautifulSoup(response.text,'html.parser')

# Get pagination links 

In [25]:
# There are 153 pages, starting from index page 1
pagination_html = soup.find('div',{'class':'css-95dbty'}).find_all('button',{'class':'temporaryButtonClassname'})
for i in range(len(pagination_html)):
    print(pagination_html[i].get_text())

1
2
3
4
5
153


In [19]:
# We can see that when we choose the 2nd page then the offset attribute in the url changes 2nd page -> offset = 18 (or [2-1]*18,
# 3rd page -> offset = 36 (or [3-1]*18) etc)

In [28]:
links_of_pages = []
for i in range(1,int(pagination_html[-1].get_text())+1):
    if i == 1:
        links_of_pages.append('https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/')
    else:
        links_of_pages.append('https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset={}'.format((i-1)*18))

In [29]:
links_of_pages

['https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=18',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=36',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=54',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=72',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=90',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=108',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=126',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=144',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=162',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=180',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset=198',
 'https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?of

# Get links of ads in first page 

In [55]:
# first page contains 18 ads 
first_page_ads = soup.find_all('div',{'class':'css-7r8xmo'})[1].find_all('div',{'class':'temporaryFlexColumnClassName css-nkly31'})
len(first_page_ads)

18

In [51]:
soup.find_all('div',{'class':'css-7r8xmo'})[1].find_all('div',{'class':'temporaryFlexColumnClassName css-nkly31'})[0].find('div',{'class':'css-m2wmpr'}).find('a')['href']

'/en/rental-apartments/k%C3%B8benhavn/113m2-3-room-id-5427608'

In [58]:
first_page_links = []
for i in range(len(first_page_ads)):
    first_page_links.append('https://www.boligportal.dk{}'.format(first_page_ads[i].find('div',{'class':'css-m2wmpr'}).find('a')['href']))

In [59]:
first_page_links

['https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/113m2-3-room-id-5427608',
 'https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/96m2-2-room-id-5427607',
 'https://www.boligportal.dk/en/rental-rooms/k%C3%B8benhavn/7m2-1-room-id-5427192',
 'https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/117m2-4-room-id-5427605',
 'https://www.boligportal.dk/en/rental-rooms/k%C3%B8benhavn/13m2-1-room-id-1556517',
 'https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/111m2-4-room-id-4651675',
 'https://www.boligportal.dk/en/rental-rooms/k%C3%B8benhavn/12m2-1-room-id-5427287',
 'https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/102m2-2-room-id-5223900',
 'https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/42m2-1-room-id-5427588',
 'https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/84m2-2-room-id-5427586',
 'https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/127m2-4-room-id-5258148',
 'https://www.boligpo

# Lets get the links of all ads across all pages 

In [60]:
# So, now we have to iterate through links_of_pages list, connect to each link and get its links.

In [212]:
all_ads_links = []
for link in tqdm(links_of_pages):
    url = link
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    page_ads = soup.find_all('div',{'class':'css-7r8xmo'})[1].find_all('div',{'class':'temporaryFlexColumnClassName css-nkly31'})
    for i in range(len(page_ads)):
        all_ads_links.append('https://www.boligportal.dk{}'.format(page_ads[i].find('div',{'class':'css-m2wmpr'}).find('a')['href']))

100%|████████████████████████████████████████████████████████████████████████████████| 153/153 [01:32<00:00,  1.65it/s]


In [213]:
len(all_ads_links)

2741

# Get data

In [215]:
# Now I have to connect to each link from above and get the data that I need. 
# Store them in a dictionary and append them in a list . 
# Should look like this [{home1:id},{home2:id}, etc]

In [216]:
#########

In [217]:
url = "https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/96m2-2-room-id-5427607"
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

In [218]:
# adress
soup.find_all('div',{'class':'css-1gjufnd'})[0].find('div',{'class':'css-o9y6d5'}).get_text()

'Nordmarks Alle, 2620 København, Albertslund  - 2nd floor'

In [219]:
# About property 
about_property = soup.find_all('section',{'class':'css-6tqm96'})[0]

In [220]:
# Property tpy
about_property.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[0].find('div',{'class':'css-6tqm96'}).get_text()

'Property type'

In [221]:
about_property.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[0].find('div',{'class':'css-5fqrc1'}).get_text()

'Apartment'

In [222]:
lst1 = []
lst2 = []
for i in range(len(about_property.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'}))):
    lst1.append(about_property.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[i].find('div',{'class':'css-6tqm96'}).get_text())
    lst2.append(about_property.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[i].find('div',{'class':'css-5fqrc1'}).get_text())
    new = dict(zip(lst1,lst2))

In [223]:
new

{'Property type': 'Apartment',
 'Size': '96 m²',
 'Rooms': '2',
 'Floor': '2nd ',
 'Furnished': 'No',
 'Shareable': 'No',
 'Pets allowed': 'Yes',
 'Elevator': 'Yes',
 'Senior friendly': 'Not specified',
 'Students only': 'Not specified',
 'Balcony': 'Yes',
 'Parking': 'Not specified',
 'Dishwasher': 'Not specified',
 'Washing machine': 'Not specified',
 'Electric charging station': 'No',
 'Dryer': 'Not specified',
 'Energy rating': '-'}

In [224]:
# About rental 
about_rental = soup.find_all('section',{'class':'css-6tqm96'})[1]

In [225]:
about_rental.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[0].find('div',{'class':'css-6tqm96'}).get_text()

'Rental period'

In [226]:
about_rental.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[0].find('div',{'class':'css-5fqrc1'}).get_text()

'Unlimited'

In [227]:
lst2 = []
lst3 = []
for i in range(len(about_rental.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'}))):
    lst2.append(about_rental.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[i].find('div',{'class':'css-6tqm96'}).get_text())
    lst3.append(about_rental.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[i].find('div',{'class':'css-5fqrc1'}).get_text())
    new1 = dict(zip(lst2,lst3))

In [228]:
new1

{'Rental period': 'Unlimited',
 'Available from': 'As soon as possible',
 'Monthly net rent': '12.400 kr.',
 'Deposit': '37.200 kr.',
 'Prepaid rent': '12.400 kr.',
 'Move-in price': '62.000 kr.',
 'Creation Date': '21/03/2024',
 'Listing-id': '5427607'}

In [229]:
new.update(new1)

In [230]:
new['Adress'] = soup.find_all('div',{'class':'css-1gjufnd'})[0].find('div',{'class':'css-o9y6d5'}).get_text()

In [231]:
new

{'Property type': 'Apartment',
 'Size': '96 m²',
 'Rooms': '2',
 'Floor': '2nd ',
 'Furnished': 'No',
 'Shareable': 'No',
 'Pets allowed': 'Yes',
 'Elevator': 'Yes',
 'Senior friendly': 'Not specified',
 'Students only': 'Not specified',
 'Balcony': 'Yes',
 'Parking': 'Not specified',
 'Dishwasher': 'Not specified',
 'Washing machine': 'Not specified',
 'Electric charging station': 'No',
 'Dryer': 'Not specified',
 'Energy rating': '-',
 'Rental period': 'Unlimited',
 'Available from': 'As soon as possible',
 'Monthly net rent': '12.400 kr.',
 'Deposit': '37.200 kr.',
 'Prepaid rent': '12.400 kr.',
 'Move-in price': '62.000 kr.',
 'Creation Date': '21/03/2024',
 'Listing-id': '5427607',
 'Adress': 'Nordmarks Alle, 2620 København, Albertslund  - 2nd floor'}

In [232]:
# Now I have to connect to each link from above and get the data that I need. 
# Store them in a dictionary and append them in a list . 
# Should look like this [{home1:id},{home2:id}, etc]

In [233]:
def get_data(link):
    prop_key = []
    prop_value = []
    rental_key = []
    rental_value = []
    properties = {}
    rental_features = {}
    
    url = link
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    about_property = soup.find_all('section',{'class':'css-6tqm96'})[0]
    about_rental = soup.find_all('section',{'class':'css-6tqm96'})[1]

    for i in range(len(about_property.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'}))):
        prop_key.append(about_property.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[i].find('div',{'class':'css-6tqm96'}).get_text())
        prop_value.append(about_property.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[i].find('div',{'class':'css-5fqrc1'}).get_text())
        properties = dict(zip(prop_key,prop_value))

    for i in range(len(about_rental.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'}))):
        rental_key.append(about_rental.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[i].find('div',{'class':'css-6tqm96'}).get_text())
        rental_value.append(about_rental.find_all('div',{'class':'css-1f8ckkp'})[0].find_all('div',{'class':'temporaryFlexColumnClassName css-1ksgrzt'})[i].find('div',{'class':'css-5fqrc1'}).get_text())
        rental_features = dict(zip(rental_key,rental_value))
    properties.update(rental_features)
    properties['Adress'] = soup.find_all('div',{'class':'css-1gjufnd'})[0].find('div',{'class':'css-o9y6d5'}).get_text()
    return properties

In [234]:
data = [] 
for link in tqdm(all_ads_links):
    data.append(get_data(link))

100%|██████████████████████████████████████████████████████████████████████████████| 2741/2741 [20:00<00:00,  2.28it/s]


In [239]:
data

[{'Property type': 'Apartment',
  'Size': '100 m²',
  'Rooms': '4',
  'Floor': 'Ground floor',
  'Furnished': 'No',
  'Shareable': 'Yes',
  'Pets allowed': 'No',
  'Elevator': 'No',
  'Senior friendly': 'No',
  'Students only': 'No',
  'Balcony': 'No',
  'Parking': 'No',
  'Dishwasher': 'Not specified',
  'Washing machine': 'Not specified',
  'Electric charging station': 'Not specified',
  'Dryer': 'Not specified',
  'Energy rating': '',
  'Rental period': '1-11 months',
  'Available from': '1 September 2024',
  'Monthly net rent': '22.000 kr.',
  'Utilities': '0 kr.',
  'Deposit': '22.000 kr.',
  'Prepaid rent': '22.000 kr.',
  'Move-in price': '66.000 kr.',
  'Creation Date': '21/03/2024',
  'Listing-id': '5427722',
  'Adress': 'Løgstørgade, 2100 København, København Ø  - Ground floor'},
 {'Property type': 'Apartment',
  'Size': '154 m²',
  'Rooms': '5',
  'Floor': '1st ',
  'Furnished': 'No',
  'Shareable': 'No',
  'Pets allowed': 'No',
  'Elevator': 'No',
  'Senior friendly': 'No',

# Load data to a dataframe

In [240]:
df = pd.DataFrame(data)

In [241]:
df

,Property type,Size,Rooms,Floor,Furnished,Shareable,Pets allowed,Elevator,Senior friendly,Students only,...,Rental period,Available from,Monthly net rent,Utilities,Deposit,Prepaid rent,Move-in price,Creation Date,Listing-id,Adress
0,Apartment,100 m²,4,Ground floor,No,Yes,No,No,No,No,...,1-11 months,1 September 2024,22.000 kr.,0 kr.,22.000 kr.,22.000 kr.,66.000 kr.,21/03/2024,5427722,"Løgstørgade, 2100 København, København Ø - Gr..."
1,Apartment,154 m²,5,1st,No,No,No,No,No,No,...,Unlimited,1 May 2024,23.000 kr.,1.400 kr.,69.000 kr.,69.000 kr.,162.400 kr.,21/03/2024,5427716,"Virginiavej, 2000 København, Frederiksberg - ..."
2,Apartment,54 m²,2,Ground floor,No,No,No,No,No,No,...,Unlimited,As soon as possible,8.448 kr.,410 kr.,25.344 kr.,25.344 kr.,59.546 kr.,21/03/2024,5427711,"Ellensvej, 2920 København, Charlottenlund - G..."
3,Apartment,71 m²,3,5th,No,Yes,Yes,Yes,No,No,...,Unlimited,As soon as possible,14.300 kr.,700 kr.,42.900 kr.,14.300 kr.,72.200 kr.,21/03/2024,5427710,"Martha Christensens Vej, 2300 København, Køben..."
4,Apartment,63 m²,2,4th,No,Yes,Yes,Yes,No,No,...,Unlimited,As soon as possible,12.700 kr.,650 kr.,38.100 kr.,12.700 kr.,64.150 kr.,21/03/2024,5427709,"Martha Christensens Vej, 2300 København, Køben..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2736,Apartment,55 m²,2,Ground floor,Yes,No,No,No,Yes,No,...,1-11 months,15 July 2024,14.000 kr.,0 kr.,2.000 kr.,0 kr.,16.000 kr.,27/12/2021,5218308,"Hedemannsgade, 2100 København, København Ø - ..."
2737,Room,15 m²,1,3rd,Yes,Yes,Yes,No,No,No,...,1-11 months,14 April 2024,8.000 kr.,0 kr.,10.000 kr.,0 kr.,18.000 kr.,17/07/2021,5091127,"Marstalsgade, 2100 København, København Ø - 3..."
2738,Apartment,125 m²,3,4th,Yes,Yes,No,No,No,No,...,1-11 months,As soon as possible,30.000 kr.,0 kr.,10.000 kr.,0 kr.,40.000 kr.,07/07/2021,5204260,"Arkonagade, 1726 København, København V - 4th..."
2739,Room,22 m²,1,4th,Yes,No,No,Not specified,Not specified,Not specified,...,24+ months,As soon as possible,3.800 kr.,0 kr.,7.600 kr.,3.800 kr.,15.200 kr.,29/10/2020,5024726,"Jagtvej, 2100 København, København Ø - 4th floor"


In [244]:
df.to_csv('ads_data.csv')